### Train a biomedical NER model from scratch

In [1]:
# 1. get the corpus
from flair.data import Corpus
from flair.datasets import ColumnCorpus

columns = {0: 'text',  1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'D:/FloridaBlue/Flair/Optim/data/ner/i2b2-disease/check'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')
    
    
print(corpus)

2020-09-30 17:10:16,355 Reading data from D:\FloridaBlue\Flair\Optim\data\ner\i2b2-disease\check
2020-09-30 17:10:16,357 Train: D:\FloridaBlue\Flair\Optim\data\ner\i2b2-disease\check\train.txt
2020-09-30 17:10:16,357 Dev: D:\FloridaBlue\Flair\Optim\data\ner\i2b2-disease\check\dev.txt
2020-09-30 17:10:16,358 Test: D:\FloridaBlue\Flair\Optim\data\ner\i2b2-disease\check\test.txt
Corpus: 11129 train + 2636 dev + 2550 test sentences


In [2]:
# 2. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type="ner")

# 3. initialize embeddings
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.embeddings import TransformerWordEmbeddings

biobert_dis_embeddings = TransformerWordEmbeddings('D:/FloridaBlue/Flair/Optim/embeddings/pretrained_bert_tf/biobert-base-discharge-cased')

embedding_types = [

#     # word embeddings trained on PubMed and PMC
#     WordEmbeddings("pubmed"),

#     # flair embeddings trained on PubMed and PMC
#     FlairEmbeddings("pubmed-forward"),
#     FlairEmbeddings("pubmed-backward"),
    biobert_dis_embeddings,
]
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [3]:
# 4. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type="ner",
    use_crf=True,
    locked_dropout=0.5
)

In [4]:
# 5. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, corpus)

trainer.train(
    base_path="taggers/i2b2-disease-biodisbert",
    train_with_dev=False,
    max_epochs=200,
    learning_rate=0.1,
    mini_batch_size=16
)

2020-09-30 17:15:00,857 ----------------------------------------------------------------------------------------------------
2020-09-30 17:15:00,859 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear

2020-09-30 17:15:00,860 ----------------------------------------------------------------------------------------------------
2020-09-30 17:15:00,861 Corpus: "Corpus: 11129 train + 2636 dev + 2550 test sentences"
2020-09-30 17:15:00,862 ----------------------------------------------------------------------------------------------------
2020-09-30 17:15:00,862 Parameters:
2020-09-30 17:15:00,862  - learning_rate: "0.1"
2020-09-30 17:15:00,863  - mini_batch_size: "16"
2020-09-30 17:15:00,863  - patience: "3"
2020-09-30 17:15:00,864  - anneal_factor: "0.5"
2020-09-30 17:15:00,865  - max_epochs: "200"
2020-09-30 17:15:00,865  - shuffle: "True"
2020-09-30 17:15:00,866  - train_with_dev: "False"
2020-09-30 17:15:00,866  - batch_growth_annealing: "False"
2020-09-30 17:15:00,866 ----------------------------------------------------------------------------------------------------
2020-09-30 17:15:00,867 Model training base path: "taggers\i2b2-disease-biodisbert"
2020-09-30 17:15:00,867 ----------

2020-09-30 17:21:34,575 epoch 5 - iter 690/696 - loss 0.97177381 - samples/sec: 287.12 - lr: 0.100000
2020-09-30 17:21:34,913 ----------------------------------------------------------------------------------------------------
2020-09-30 17:21:34,913 EPOCH 5 done: loss 0.9682 - lr 0.1000000
2020-09-30 17:21:38,937 DEV : loss 0.5041687488555908 - score 0.7889
2020-09-30 17:21:38,980 BAD EPOCHS (no improvement): 0
saving best model
2020-09-30 17:21:40,393 ----------------------------------------------------------------------------------------------------
2020-09-30 17:21:44,481 epoch 6 - iter 69/696 - loss 0.95990464 - samples/sec: 270.08 - lr: 0.100000
2020-09-30 17:21:48,512 epoch 6 - iter 138/696 - loss 0.98982613 - samples/sec: 273.85 - lr: 0.100000
2020-09-30 17:21:52,484 epoch 6 - iter 207/696 - loss 1.01992827 - samples/sec: 278.00 - lr: 0.100000
2020-09-30 17:21:56,446 epoch 6 - iter 276/696 - loss 0.98843733 - samples/sec: 278.61 - lr: 0.100000
2020-09-30 17:22:00,425 epoch 6 - 

2020-09-30 17:25:39,457 epoch 11 - iter 276/696 - loss 0.76414120 - samples/sec: 274.78 - lr: 0.100000
2020-09-30 17:25:43,262 epoch 11 - iter 345/696 - loss 0.77222034 - samples/sec: 290.23 - lr: 0.100000
2020-09-30 17:25:47,150 epoch 11 - iter 414/696 - loss 0.77196258 - samples/sec: 285.09 - lr: 0.100000
2020-09-30 17:25:51,145 epoch 11 - iter 483/696 - loss 0.76522480 - samples/sec: 277.42 - lr: 0.100000
2020-09-30 17:25:55,108 epoch 11 - iter 552/696 - loss 0.75376235 - samples/sec: 278.58 - lr: 0.100000
2020-09-30 17:25:59,132 epoch 11 - iter 621/696 - loss 0.75341763 - samples/sec: 274.34 - lr: 0.100000
2020-09-30 17:26:03,105 epoch 11 - iter 690/696 - loss 0.74964702 - samples/sec: 277.90 - lr: 0.100000
2020-09-30 17:26:03,527 ----------------------------------------------------------------------------------------------------
2020-09-30 17:26:03,527 EPOCH 11 done: loss 0.7500 - lr 0.1000000
2020-09-30 17:26:07,510 DEV : loss 0.44905078411102295 - score 0.8006
2020-09-30 17:26:0

2020-09-30 17:29:46,125 EPOCH 16 done: loss 0.6371 - lr 0.1000000
2020-09-30 17:29:50,098 DEV : loss 0.4635355472564697 - score 0.7984
2020-09-30 17:29:50,136 BAD EPOCHS (no improvement): 2
2020-09-30 17:29:50,137 ----------------------------------------------------------------------------------------------------
2020-09-30 17:29:54,044 epoch 17 - iter 69/696 - loss 0.61632176 - samples/sec: 282.65 - lr: 0.100000
2020-09-30 17:29:57,805 epoch 17 - iter 138/696 - loss 0.62728529 - samples/sec: 293.50 - lr: 0.100000
2020-09-30 17:30:01,954 epoch 17 - iter 207/696 - loss 0.62515833 - samples/sec: 266.27 - lr: 0.100000
2020-09-30 17:30:05,993 epoch 17 - iter 276/696 - loss 0.64346101 - samples/sec: 273.42 - lr: 0.100000
2020-09-30 17:30:09,915 epoch 17 - iter 345/696 - loss 0.60902750 - samples/sec: 281.54 - lr: 0.100000
2020-09-30 17:30:13,856 epoch 17 - iter 414/696 - loss 0.59819690 - samples/sec: 280.10 - lr: 0.100000
2020-09-30 17:30:17,904 epoch 17 - iter 483/696 - loss 0.60160580 - 

...

2020-09-30 18:02:44,409 epoch 61 - iter 414/696 - loss 0.33738756 - samples/sec: 282.40 - lr: 0.003125
2020-09-30 18:02:48,397 epoch 61 - iter 483/696 - loss 0.34366038 - samples/sec: 277.92 - lr: 0.003125
2020-09-30 18:02:52,376 epoch 61 - iter 552/696 - loss 0.33762828 - samples/sec: 277.47 - lr: 0.003125
2020-09-30 18:02:56,334 epoch 61 - iter 621/696 - loss 0.33823369 - samples/sec: 279.40 - lr: 0.003125
2020-09-30 18:03:00,213 epoch 61 - iter 690/696 - loss 0.33528626 - samples/sec: 284.63 - lr: 0.003125
2020-09-30 18:03:00,552 ----------------------------------------------------------------------------------------------------
2020-09-30 18:03:00,552 EPOCH 61 done: loss 0.3347 - lr 0.0031250
2020-09-30 18:03:04,559 DEV : loss 0.3238588869571686 - score 0.8547
2020-09-30 18:03:04,600 BAD EPOCHS (no improvement): 3
2020-09-30 18:03:04,600 ----------------------------------------------------------------------------------------------------
2020-09-30 18:03:08,677 epoch 62 - iter 69/69

2020-09-30 18:06:44,425 ----------------------------------------------------------------------------------------------------
2020-09-30 18:06:48,386 epoch 67 - iter 69/696 - loss 0.35426759 - samples/sec: 278.73 - lr: 0.000781
2020-09-30 18:06:52,080 epoch 67 - iter 138/696 - loss 0.33416006 - samples/sec: 298.90 - lr: 0.000781
2020-09-30 18:06:56,159 epoch 67 - iter 207/696 - loss 0.33333932 - samples/sec: 270.64 - lr: 0.000781
2020-09-30 18:07:00,185 epoch 67 - iter 276/696 - loss 0.33650363 - samples/sec: 274.23 - lr: 0.000781
2020-09-30 18:07:04,027 epoch 67 - iter 345/696 - loss 0.33813471 - samples/sec: 287.38 - lr: 0.000781
2020-09-30 18:07:07,951 epoch 67 - iter 414/696 - loss 0.33594972 - samples/sec: 281.29 - lr: 0.000781
2020-09-30 18:07:11,861 epoch 67 - iter 483/696 - loss 0.33020006 - samples/sec: 282.41 - lr: 0.000781
2020-09-30 18:07:15,755 epoch 67 - iter 552/696 - loss 0.33557666 - samples/sec: 283.49 - lr: 0.000781
2020-09-30 18:07:20,070 epoch 67 - iter 621/696 - lo

2020-09-30 18:10:56,401 epoch 72 - iter 552/696 - loss 0.34613475 - samples/sec: 289.78 - lr: 0.000391
2020-09-30 18:11:00,295 epoch 72 - iter 621/696 - loss 0.34393900 - samples/sec: 283.48 - lr: 0.000391
2020-09-30 18:11:04,121 epoch 72 - iter 690/696 - loss 0.34466745 - samples/sec: 288.55 - lr: 0.000391
2020-09-30 18:11:04,421 ----------------------------------------------------------------------------------------------------
2020-09-30 18:11:04,421 EPOCH 72 done: loss 0.3438 - lr 0.0003906
2020-09-30 18:11:08,403 DEV : loss 0.3240659534931183 - score 0.8498
2020-09-30 18:11:08,443 BAD EPOCHS (no improvement): 2
2020-09-30 18:11:08,444 ----------------------------------------------------------------------------------------------------
2020-09-30 18:11:12,517 epoch 73 - iter 69/696 - loss 0.34190644 - samples/sec: 271.09 - lr: 0.000391
2020-09-30 18:11:16,368 epoch 73 - iter 138/696 - loss 0.35269239 - samples/sec: 286.71 - lr: 0.000391
2020-09-30 18:11:20,279 epoch 73 - iter 207/69

2020-09-30 18:14:55,391 epoch 78 - iter 138/696 - loss 0.34422275 - samples/sec: 285.52 - lr: 0.000195
2020-09-30 18:14:59,385 epoch 78 - iter 207/696 - loss 0.33206080 - samples/sec: 276.39 - lr: 0.000195
2020-09-30 18:15:03,110 epoch 78 - iter 276/696 - loss 0.33118748 - samples/sec: 296.89 - lr: 0.000195
2020-09-30 18:15:07,321 epoch 78 - iter 345/696 - loss 0.32469978 - samples/sec: 263.17 - lr: 0.000195
2020-09-30 18:15:11,462 epoch 78 - iter 414/696 - loss 0.32701788 - samples/sec: 266.60 - lr: 0.000195
2020-09-30 18:15:15,440 epoch 78 - iter 483/696 - loss 0.33381588 - samples/sec: 277.55 - lr: 0.000195
2020-09-30 18:15:19,179 epoch 78 - iter 552/696 - loss 0.33172036 - samples/sec: 296.45 - lr: 0.000195
2020-09-30 18:15:23,227 epoch 78 - iter 621/696 - loss 0.32934404 - samples/sec: 272.75 - lr: 0.000195
2020-09-30 18:15:27,183 epoch 78 - iter 690/696 - loss 0.33103595 - samples/sec: 279.04 - lr: 0.000195
2020-09-30 18:15:27,499 -------------------------------------------------

{'test_score': 0.8505846686877436,
 'dev_score_history': [0.5931398416886543,
  0.7528409090909091,
  0.7051975051975051,
  0.7790228597041685,
  0.7889492753623187,
  0.7445699391833189,
  0.8018433179723502,
  0.7756756756756757,
  0.8042532624456259,
  0.7928938756428237,
  0.8005850804485617,
  0.7866149369544132,
  0.8058712121212122,
  0.8165840468679585,
  0.7840290381125228,
  0.7984084880636605,
  0.8133971291866029,
  0.8027272727272727,
  0.8110091743119265,
  0.8199162401116797,
  0.8180163785259326,
  0.8338762214983713,
  0.819926873857404,
  0.8383510884668829,
  0.8274907749077489,
  0.838147638697845,
  0.8220183486238533,
  0.8346883468834688,
  0.8283378746594006,
  0.8413145539906105,
  0.8334106728538283,
  0.813634351482957,
  0.8361478776814241,
  0.8457943925233644,
  0.8353715898400752,
  0.8365601080594328,
  0.8274616079494128,
  0.8310871518418688,
  0.848089468779124,
  0.8478964401294499,
  0.8484848484848484,
  0.8469387755102041,
  0.8533710513908533,
  

In [ ]:
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger, MultiTagger

#data_folder = 'D:/FloridaBlue/Flair/Optim/data/ner/i2b2-disease/check'
data_folder = 'D:/FloridaBlue/Flair/Optim/data/ner/NCBI-disease'
corpus: ColumnCorpus = ColumnCorpus(data_folder, column_format={0: 'text', 3: 'ner'})


#tagger: SequenceTagger = SequenceTagger.load("taggers/i2b2-disease-flair-word2/best-model.pt")
tagger: SequenceTagger = SequenceTagger.load("taggers/ncbi-disease-flair-word-readdataset/best-model.pt")
#tagger: SequenceTagger = SequenceTagger.load("hunflair-disease")
#tagger = MultiTagger.load("hunflair-chemical")
result, _ = tagger.evaluate(corpus.test)
print(result.detailed_results)

In [ ]:
SequenceTagger.evaluate()